<a href="https://colab.research.google.com/github/plaban1981/Medical_paper_summarization/blob/main/Medical_Domain_Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install --quiet transformers

In [3]:
! pip install --quiet sentencepiece

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline


In [5]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('stopwords')
nltk.download('brown')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:

tokenizer = AutoTokenizer.from_pretrained("L-macc/autotrain-Biomedical_sc_summ-1217846148")

model = AutoModelForSeq2SeqLM.from_pretrained("L-macc/autotrain-Biomedical_sc_summ-1217846148")

In [ ]:
import torch 
from transformers import pipeline
device = "cuda" if torch.cuda.is_available() else "cpu"
summarizer = pipeline("summarization", model='facebook/bart-large-cnn',min_length=75,max_length=500)

In [ ]:
text = """Telemedicine and health policy: A systematic review

Highlights
Frequently mentioned facilitators were access, convenience, population health, mobile technology care, and self-efficacy.
Frequently mentioned barriers were the increased cost to providers, patient privacy, technical literacy, state licensing, and data security.
Health policies can increase access, improve population health, and remove constraints to reach patients in developing countries.
Abstract
Background
Telemedicine diagnoses and treats patients remotely via telecommunications technology all over the world. Telemedicine becomes more prevalent as providers recognize the benefits, patients receive increased access and payers see the reduction in cost of care.

Objective
Telemedicine studies have shown success in limiting geographical constraints, time spent, and costs incurred by patients with positive health outcomes across medical specialties. The aim of this review is to evaluate the implications of telemedicine and health policies.

Methods
An assessment of the literature in four databases was made on content germane to health policy implications of telemedicine. From the results of the search, 48 publications were kept for analysis.

Results
The fifteen facilitators mentioned most often were increased access, increased convenience, improved population health, care enabled through mobile technology, self-efficacy, increased patient-to-provider communication, cost advantages, efficacy of modality, increased health outcomes, reaches developing countries, increased quality, a positive previous experience, and a secure means of care. The twelve barriers mentioned most often were the increased cost to providers, patient privacy, technical literacy, state licensing, data security, socioeconomics, limited reimbursements, issues of interoperability, patient safety, less personal means of care, misaligned incentives, and ethical concerns.

Conclusions
Telemedicine has the potential for growth and adoption, however, there are several implications and barriers of health policy surrounding telemedicine that make it difficult to adopt. Policies will likely encourage and incentivize its spread and use. Future research should focus on standardization of telemedicine and new policies and incentives that encourage its use.
"""

In [ ]:
summarizer(text,truncation=True)

Your max_length is set to 500, but you input_length is only 437. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=218)


[{'summary_text': 'Health policies can increase access, improve population health, and remove constraints to reach patients in developing countries. The twelve barriers mentioned most often were the increased cost to providers, patient privacy, technical literacy, state licensing, data security, socioeconomics, limited reimbursements. Future research should focus on standardization of telemedicine and new policies and incentives that encourage its use.'}]

In [ ]:
preprocessed_text = text.strip().replace("\n","")

In [15]:
preprocessed_text

'Telemedicine and health policy: A systematic reviewHighlightsFrequently mentioned facilitators were access, convenience, population health, mobile technology care, and self-efficacy.Frequently mentioned barriers were the increased cost to providers, patient privacy, technical literacy, state licensing, and data security.Health policies can increase access, improve population health, and remove constraints to reach patients in developing countries.AbstractBackgroundTelemedicine diagnoses and treats patients remotely via telecommunications technology all over the world. Telemedicine becomes more prevalent as providers recognize the benefits, patients receive increased access and payers see the reduction in cost of care.ObjectiveTelemedicine studies have shown success in limiting geographical constraints, time spent, and costs incurred by patients with positive health outcomes across medical specialties. The aim of this review is to evaluate the implications of telemedicine and health po

In [13]:
input_text = "summarize:"+preprocessed_text

In [14]:
len(input_text.split())

288

In [ ]:
torch.cuda.is_available()

True

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [9]:
tokenized_text = tokenizer.encode(input_text,return_tensors="pt",max_length=1024,truncation=True)

In [10]:
summary = model.generate(tokenized_text,min_length=30,max_length=500)
summary_text = tokenizer.decode(summary[0],skip_special_tokens=True)

/usr/local/lib/python3.8/dist-packages/transformers/modeling_utils.py:764: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [11]:
".".join([text.strip().capitalize() for text in summary_text.split(".")])

'Telemedicine has the potential for growth and adoption, however, there are several implications and barriers of health policy surrounding tele medicine that make it difficult to adopt.Policies will likely encourage and incentivize its spread and use.Future research should focus on standardization of-telemedicine and new policies and incentives that encourage its use.We conducted a systematic review of literature on content germane to health policy implications'

In [ ]:
!pip install -q bert-extractive-summarizer

In [ ]:
import spacy
import textwrap
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest
punctuation += '\n' 
stopwords = list(STOP_WORDS)

reduction_rate = 0.1  #defines how small the output summary should be compared with the input
nlp_pl = spacy.load('en_core_web_sm')     #process original text according with the Spacy nlp pipeline for english


In [ ]:
document = nlp_pl(preprocessed_text)                   #doc object
sentence_tokens = [sent for sent in document.sents]

In [ ]:
len(sentence_tokens)

14

In [ ]:
tokens = [token.text for token in document] #tokenized text

word_frequencies = {}
for word in document:
    if word.text.lower() not in stopwords:
        if word.text.lower() not in punctuation:
            if word.text not in word_frequencies.keys():
                word_frequencies[word.text] = 1
            else:
                word_frequencies[word.text] += 1

max_frequency = max(word_frequencies.values())
print(max_frequency)

for word in word_frequencies.keys():
    word_frequencies[word] = word_frequencies[word]/max_frequency

print(word_frequencies)

9
{'Telemedicine': 0.2222222222222222, 'health': 1.0, 'policy': 0.3333333333333333, 'systematic': 0.1111111111111111, 'reviewHighlightsFrequently': 0.1111111111111111, 'mentioned': 0.4444444444444444, 'facilitators': 0.2222222222222222, 'access': 0.4444444444444444, 'convenience': 0.2222222222222222, 'population': 0.3333333333333333, 'mobile': 0.2222222222222222, 'technology': 0.3333333333333333, 'care': 0.5555555555555556, 'self': 0.2222222222222222, 'efficacy': 0.3333333333333333, 'Frequently': 0.1111111111111111, 'barriers': 0.3333333333333333, 'increased': 0.8888888888888888, 'cost': 0.4444444444444444, 'providers': 0.3333333333333333, 'patient': 0.4444444444444444, 'privacy': 0.2222222222222222, 'technical': 0.2222222222222222, 'literacy': 0.2222222222222222, 'state': 0.2222222222222222, 'licensing': 0.2222222222222222, 'data': 0.2222222222222222, 'security': 0.2222222222222222, 'Health': 0.1111111111111111, 'policies': 0.3333333333333333, 'increase': 0.1111111111111111, 'improve'

In [ ]:
def get_sentence_scores(sentence_tok, len_norm=True):
  sentence_scores = {}
  for sent in sentence_tok:
      word_count = 0
      for word in sent:
          if word.text.lower() in word_frequencies.keys():
              word_count += 1
              if sent not in sentence_scores.keys():
                  sentence_scores[sent] = word_frequencies[word.text.lower()]
              else:
                  sentence_scores[sent] += word_frequencies[word.text.lower()]
      if len_norm:
        sentence_scores[sent] = sentence_scores[sent]/word_count
  return sentence_scores
                
sentence_scores = get_sentence_scores(sentence_tokens,len_norm=False)        #sentence scoring without lenght normalization
sentence_scores_rel = get_sentence_scores(sentence_tokens,len_norm=True)

In [ ]:
def get_summary(sentence_sc, rate):
  summary_length = int(len(sentence_sc)*rate)
  summary = nlargest(summary_length, sentence_sc, key = sentence_sc.get)
  final_summary = [word.text for word in summary]
  summary = ' '.join(final_summary)
  return summary
#the first output shows verbose and content-heavy sentences, while the second one is much more concise.
print("- NON_REL: "+ get_summary(sentence_scores, reduction_rate))
print("- REL: "+ get_summary(sentence_scores_rel, reduction_rate))

- NON_REL: ResultsThe fifteen facilitators mentioned most often were increased access, increased convenience, improved population health, care enabled through mobile technology, self-efficacy, increased patient-to-provider communication, cost advantages, efficacy of modality, increased health outcomes, reaches developing countries, increased quality, a positive previous experience, and a secure means of care.
- REL: Telemedicine and health policy: A systematic reviewHighlightsFrequently mentioned facilitators were access, convenience, population health, mobile technology care, and self-efficacy.


In [ ]:
from summarizer import Summarizer
model = Summarizer()

summary_length = int(len(sentence_tokens)*reduction_rate)
print(summary_length)
result = model(preprocessed_text, num_sentences=summary_length, min_length=400)

print(''.join(result))

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


1
ResultsThe fifteen facilitators mentioned most often were increased access, increased convenience, improved population health, care enabled through mobile technology, self-efficacy, increased patient-to-provider communication, cost advantages, efficacy of modality, increased health outcomes, reaches developing countries, increased quality, a positive previous experience, and a secure means of care.


In [ ]:
result

'ResultsThe fifteen facilitators mentioned most often were increased access, increased convenience, improved population health, care enabled through mobile technology, self-efficacy, increased patient-to-provider communication, cost advantages, efficacy of modality, increased health outcomes, reaches developing countries, increased quality, a positive previous experience, and a secure means of care.'

In [ ]:
result = model(preprocessed_text,min_length=400)

print(''.join(result.replace('Results',"")))

The fifteen facilitators mentioned most often were increased access, increased convenience, improved population health, care enabled through mobile technology, self-efficacy, increased patient-to-provider communication, cost advantages, efficacy of modality, increased health outcomes, reaches developing countries, increased quality, a positive previous experience, and a secure means of care.


In [ ]:
from summarizer import TransformerSummarizer
GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")
full = ''.join(GPT2_model(preprocessed_text, min_length=60))
print(full)

Telemedicine and health policy: A systematic reviewHighlightsFrequently mentioned facilitators were access, convenience, population health, mobile technology care, and self-efficacy. ObjectiveTelemedicine studies have shown success in limiting geographical constraints, time spent, and costs incurred by patients with positive health outcomes across medical specialties. Policies will likely encourage and incentivize its spread and use.


In [ ]:
from summarizer import TransformerSummarizer
GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")
full = ''.join(GPT2_model(preprocessed_text, min_length=60))
print(full)

Telemedicine and health policy: A systematic reviewHighlightsFrequently mentioned facilitators were access, convenience, population health, mobile technology care, and self-efficacy. ObjectiveTelemedicine studies have shown success in limiting geographical constraints, time spent, and costs incurred by patients with positive health outcomes across medical specialties. Policies will likely encourage and incentivize its spread and use.


In [ ]:
from summarizer import TransformerSummarizer
model = TransformerSummarizer(transformer_type="XLNet",transformer_model_key="xlnet-base-cased")
full = ''.join(model(preprocessed_text, min_length=300))
print(full)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetModel: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


ResultsThe fifteen facilitators mentioned most often were increased access, increased convenience, improved population health, care enabled through mobile technology, self-efficacy, increased patient-to-provider communication, cost advantages, efficacy of modality, increased health outcomes, reaches developing countries, increased quality, a positive previous experience, and a secure means of care. The twelve barriers mentioned most often were the increased cost to providers, patient privacy, technical literacy, state licensing, data security, socioeconomics, limited reimbursements, issues of interoperability, patient safety, less personal means of care, misaligned incentives, and ethical concerns.


In [ ]:
preprocessed_text = text.strip().replace("\n","")

In [ ]:
from summarizer import TransformerSummarizer
GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")
full = ''.join(GPT2_model(preprocessed_text, min_length=400))

In [ ]:
print(''.join(full.replace('Results',"")))

Although abstractive summariesare generally more beneficial to researchers because summaries used2M.K. Rohil and V. MagotraHealthcare Analytics 2 (2022) 100058for research work must highlight multiple and relevant aspects of thesemantics of the document, most of the existing methods produce ex-tractive summaries which are easier to generate because the documentthat needs to be summarized is aligned with a single aspect. These methods use different approaches suchas statistical approach, natural language processing approach, ma-chine learning approach, graph-based approach [4,5,12] and algebraicapproach amongst others [5].As documents in the biomedical and healthcare domains have dif-ferent characteristics and features, depending on the type of documents3M.K. Rohil and V. MagotraHealthcare Analytics 2 (2022) 100058Table 3Summary of text summarization related research and applications to the biomedical and healthcare domain.


## Reading PDF

In [6]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [7]:
! pip install PyMuPDF

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
type(doc)


fitz.fitz.Document

https://haystack.deepset.ai/tutorials/16_document_classifier_at_index_time

##PDF - Extracting Text

In [13]:
import fitz
from operator import itemgetter

# Load the PDF document using PyMuPDF
doc = fitz.open("/content/Published.SAFPJ.11.05.2017.pdf")


## Extract Headers and paragraphs from PDF

In [8]:
def fonts(doc, granularity=False):
    """Extracts fonts and their usage in PDF documents.
    :param doc: PDF document to iterate through
    :type doc: <class 'fitz.fitz.Document'>
    :param granularity: also use 'font', 'flags' and 'color' to discriminate text
    :type granularity: bool
    :rtype: [(font_size, count), (font_size, count}], dict
    :return: most used fonts sorted by count, font style information
    """
    styles = {}
    font_counts = {}

    for page in doc:
        blocks = page.get_text("dict")["blocks"]
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # block contains text
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        if granularity:
                            identifier = "{0}_{1}_{2}_{3}".format(s['size'], s['flags'], s['font'], s['color'])
                            styles[identifier] = {'size': s['size'], 'flags': s['flags'], 'font': s['font'],
                                                  'color': s['color']}
                        else:
                            identifier = "{0}".format(s['size'])
                            styles[identifier] = {'size': s['size'], 'font': s['font']}

                        font_counts[identifier] = font_counts.get(identifier, 0) + 1  # count the fonts usage

    font_counts = sorted(font_counts.items(), key=itemgetter(1), reverse=True)

    if len(font_counts) < 1:
        raise ValueError("Zero discriminating fonts found!")

    return font_counts, styles

In [ ]:
print(dir(doc))

['FontInfos', 'FormFonts', 'Graftmaps', 'InsertedImages', 'ShownPages', '__class__', '__contains__', '__del__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__swig_destroy__', '__weakref__', '_addFormFont', '_cleanup', '_delToC', '_deleteObject', '_delete_page', '_do_links', '_dropOutline', '_embeddedFileGet', '_embeddedFileIndex', '_embfile_add', '_embfile_del', '_embfile_info', '_embfile_names', '_embfile_upd', '_extend_toc_items', '_forget_page', '_getMetadata', '_getOLRootNumber', '_getPDFfileid', '_getPageInfo', '_get_char_widths', '_get_page_labels', '_graft_id', '_insert_font', '_loadOutline', '_make_page_map', '_move_copy_page', '_newPage', '_out

In [ ]:
for page in doc:
  print(dir(page.get_text))
  break

['__call__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__func__', '__ge__', '__get__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__self__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


In [56]:
font_counts, styles = fonts(doc, granularity=False)

In [ ]:
styles

{'5.963251113891602': {'size': 5.963251113891602,
  'font': 'SourceSansPro-Semibold'},
 '13.251668930053711': {'size': 13.251668930053711, 'font': 'Martel-Regular'},
 '6.6258344650268555': {'size': 6.6258344650268555,
  'font': 'SourceSansPro-Regular'},
 '4.638084411621094': {'size': 4.638084411621094,
  'font': 'SourceSansPro-Regular'},
 '5.300667762756348': {'size': 5.300667762756348,
  'font': 'SourceSansPro-Regular'},
 '7.9510016441345215': {'size': 7.9510016441345215,
  'font': 'SourceSansPro-Regular'},
 '8.0': {'size': 8.0, 'font': 'MyriadPro-Light'},
 '7.589466094970703': {'size': 7.589466094970703, 'font': 'MyriadPro-Regular'},
 '10.0': {'size': 10.0, 'font': 'MyriadPro-Bold'},
 '7.0': {'size': 7.0, 'font': 'MyriadPro-Regular'},
 '8.81816291809082': {'size': 8.81816291809082, 'font': 'MyriadPro-Regular'},
 '5.290897846221924': {'size': 5.290897846221924, 'font': 'MyriadPro-Regular'},
 '6.453680992126465': {'size': 6.453680992126465, 'font': 'MyriadPro-Regular'},
 '18.0': {'size

In [ ]:
font_counts

[('8.81816291809082', 310),
 ('8.0', 269),
 ('7.375635623931885', 85),
 ('5.290897846221924', 40),
 ('6.6258344650268555', 26),
 ('10.0', 18),
 ('5.300667762756348', 9),
 ('7.589466094970703', 7),
 ('5.963251113891602', 6),
 ('7.0', 6),
 ('6.453680992126465', 6),
 ('9.0', 5),
 ('7.9510016441345215', 4),
 ('5.400000095367432', 2),
 ('13.251668930053711', 1),
 ('4.638084411621094', 1),
 ('18.0', 1),
 ('4.0', 1)]

In [9]:
def font_tags(font_counts, styles):
    """Returns dictionary with font sizes as keys and tags as value.
    :param font_counts: (font_size, count) for all fonts occuring in document
    :type font_counts: list
    :param styles: all styles found in the document
    :type styles: dict
    :rtype: dict
    :return: all element tags based on font-sizes
    """
    p_style = styles[font_counts[0][0]]  # get style for most used font by count (paragraph)
    p_size = p_style['size']  # get the paragraph's size

    # sorting the font sizes high to low, so that we can append the right integer to each tag
    font_sizes = []
    for (font_size, count) in font_counts:
        font_sizes.append(float(font_size))
    font_sizes.sort(reverse=True)

    # aggregating the tags for each font size
    idx = 0
    size_tag = {}
    for size in font_sizes:
        idx += 1
        if size == p_size:
            idx = 0
            size_tag[size] = '<p>'
        if size > p_size:
            size_tag[size] = '<h{0}>'.format(idx)
        elif size < p_size:
            size_tag[size] = '<s{0}>'.format(idx)

    return size_tag

In [58]:
size_tag = font_tags(font_counts, styles)

In [59]:
size_tag

{18.0: '<h1>',
 13.251668930053711: '<h2>',
 10.0: '<h3>',
 9.0: '<h4>',
 8.81816291809082: '<p>',
 8.0: '<s1>',
 7.9510016441345215: '<s2>',
 7.589466094970703: '<s3>',
 7.375635623931885: '<s4>',
 7.0: '<s5>',
 6.6258344650268555: '<s6>',
 6.453680992126465: '<s7>',
 5.963251113891602: '<s8>',
 5.400000095367432: '<s9>',
 5.300667762756348: '<s10>',
 5.290897846221924: '<s11>',
 4.638084411621094: '<s12>',
 4.0: '<s13>'}

In [10]:
def headers_para(doc, size_tag):
    """Scrapes headers & paragraphs from PDF and return texts with element tags.
    :param doc: PDF document to iterate through
    :type doc: <class 'fitz.fitz.Document'>
    :param size_tag: textual element tags for each size
    :type size_tag: dict
    :rtype: list
    :return: texts with pre-prended element tags
    """
    header_para = []  # list with headers and paragraphs
    first = True  # boolean operator for first header
    previous_s = {}  # previous span

    for page in doc:
        blocks = page.get_text("dict")["blocks"]
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # this block contains text

                # REMEMBER: multiple fonts and sizes are possible IN one block

                block_string = ""  # text found in block
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        if s['text'].strip():  # removing whitespaces:
                            if first:
                                previous_s = s
                                first = False
                                block_string = size_tag[s['size']] + s['text']
                            else:
                                if s['size'] == previous_s['size']:

                                    if block_string and all((c == "|") for c in block_string):
                                        # block_string only contains pipes
                                        block_string = size_tag[s['size']] + s['text']
                                    if block_string == "":
                                        # new block has started, so append size tag
                                        block_string = size_tag[s['size']] + s['text']
                                    else:  # in the same block, so concatenate strings
                                        block_string += " " + s['text']

                                else:
                                    header_para.append(block_string)
                                    block_string = size_tag[s['size']] + s['text']

                                previous_s = s

                    # new block started, indicating with a pipe
                    block_string += "|"

                header_para.append(block_string)

    return header_para

In [61]:
elements = headers_para(doc, size_tag)




In [62]:
type(elements)

list

In [63]:
len(elements)

453

In [64]:
elements[10]

''

##Extracting headers and paragraphs from the PDF

In [65]:
context= []
header = []
for text in elements:
    if text.startswith("<h"):
      start = text.index(">")
      header.append(text[start+1:].replace("|",""))
    if text.startswith("<p"):
      start = text.index(">")
      context.append(text[start+1:].replace("|",""))

In [71]:

header_context = []

for text in elements:
    if text.startswith("<h"):
      start = text.index(">")
      header_context.append(text[start+1:].replace("|",""))
    if text.startswith("<p"):
      start = text.index(">")
      header_context.append(text[start+1:].replace("|",""))

In [75]:
len(header_context[3])

12

## Extract text for corresponding headers

In [84]:
element_dict = {}
for text in elements:
  print(text)
  prev = text[0:start]
  if text.startswith("<h"): 
    start = text.index(">")
    header = text[start+1:].replace("|","")
    
  if text.startswith("<p") and header not in element_dict.keys():
      c = []
      start = text.index(">")
      c.append(text[start+1:].replace("|",""))
      element_dict[header] = c
  elif text.startswith("<p") and header in element_dict.keys():
      start = text.index(">")
      c.append(text[start+1:].replace("|",""))
      element_dict[header] =c
  else:
    continue
    

<s8>See discussions, stats, and author profiles for this publication at:  https://www.researchgate.net/publication/316990409|

<h2>The medical management of glaucoma|

<s6>Article in South African Family Practice · May 2017|

<s12>DOI: 10.4102/safp.v59i2.4669|

<s10>CITATIONS|
<s2>3|

<s10>READS|
<s2>11,056|

<s6>2 authors:|
<s6>Some of the authors of this publication are also working on these related projects:|
<s6>Medical View project|
<s6>Indigenous Knowledge Systems: Pharmacology and Toxicology View project|
<s6>Andre Marais|
<s6>University of Pretoria|

<s8>20
<s10>PUBLICATIONS
<s8>34
<s10>CITATIONS|
<s10>SEE PROFILE|

<s6>Elzbieta Osuch|
<s6>Sefako Makgatho Health Sciences University|

<s8>26
<s10>PUBLICATIONS
<s8>110
<s10>CITATIONS|
<s10>SEE PROFILE|

<s6>All content following this page was uploaded by  Andre Marais  on 17 May 2017.|

<s10>The user has requested enhancement of the downloaded file.|

<s1>The page number in the footer is not for bibliographic referencing| www.tand

In [36]:
element_dict.keys()

dict_keys(['Introduction', 'Abstract', 'Corresponding author, email: dramarais@gmail.com / andre.marais@up.ac.za', 'Pathophysiology of glaucoma', 'Open angle glaucoma', 'Closed angle glaucoma', ' Treatment goals', 'Pharmacological management', 'Prostaglandin receptor analogs', 'Carbonic anhydrase inhibitors', 'Alpha 2 adrenergic agonists', 'Cholinergic agonists', 'Beta Blockers', 'Combination agents', 'Cannabinoids', 'Conclusion '])

In [95]:
sent_tokenize(element_dict['Abstract'][0])

['Glaucoma is one of the leading causes of blindness worldwide.',
 'The main characteristic features include optic disc cupping, visual field  loss, and in most instances a raised intraocular pressure.',
 'Primary open angle glaucoma accounts for nearly two thirds of all cases, while  angle closure glaucoma contributes the majority of the remainder.',
 'Screening for glaucoma is not cost-effective, and diagnosis of the  primary open angle type is usually an incidental finding during ophthalmological examination.',
 'Various treatment options, including  medical and surgical interventions are accessible in the management of glaucoma.',
 'This article contains a brief description on the  pathophysiology and focuses on the pharmacological options available in the treatment of glaucoma.']

In [96]:
".".join([text.strip() for text in sent_tokenize(element_dict['Abstract'][0])])

'Glaucoma is one of the leading causes of blindness worldwide..The main characteristic features include optic disc cupping, visual field  loss, and in most instances a raised intraocular pressure..Primary open angle glaucoma accounts for nearly two thirds of all cases, while  angle closure glaucoma contributes the majority of the remainder..Screening for glaucoma is not cost-effective, and diagnosis of the  primary open angle type is usually an incidental finding during ophthalmological examination..Various treatment options, including  medical and surgical interventions are accessible in the management of glaucoma..This article contains a brief description on the  pathophysiology and focuses on the pharmacological options available in the treatment of glaucoma.'

In [67]:
final_text = " ".join(context)

In [69]:
final_text


'Glaucoma is a mixed group of eye disorders sharing common  features which typically includes optic disc cupping and visual  field loss, thereby more accurately defined as an optic neuropathy.  Optic nerve damage results in progressive retinal ganglionic  cell loss and irreversible blindness, if not treated.  Traditionally,  glaucoma has been characterized by the existence of raised  intraocular pressure (IOP), however nerve damage may occur in the  presence of values within the statistically normal range between   8–21 mmHg. Glaucomatous disc and field changes in the absence  of elevated pressure constitutes normal or low tension glaucoma  (LTG).  Similarly, conditions exceeding the upper limit of 21 mmHg  may not be associated with signs of visual field loss or optic disc  damage and comprises ocular hypertension.   In addition to optic  neuropathy, the two main types of glaucoma (open angle and  closed angle) are described on the appearance of the drainage  system within the eye, in

In [70]:
with open('Pdf2text.txt','a+') as f:
  f.write(final_text)

In [98]:
processed_text = "".join([t.strip() for t in sent_tokenize(final_text)])

In [99]:
processed_text

'Glaucoma is a mixed group of eye disorders sharing common  features which typically includes optic disc cupping and visual  field loss, thereby more accurately defined as an optic neuropathy.Optic nerve damage results in progressive retinal ganglionic  cell loss and irreversible blindness, if not treated.Traditionally,  glaucoma has been characterized by the existence of raised  intraocular pressure (IOP), however nerve damage may occur in the  presence of values within the statistically normal range between   8–21 mmHg.Glaucomatous disc and field changes in the absence  of elevated pressure constitutes normal or low tension glaucoma  (LTG).Similarly, conditions exceeding the upper limit of 21 mmHg  may not be associated with signs of visual field loss or optic disc  damage and comprises ocular hypertension.In addition to optic  neuropathy, the two main types of glaucoma (open angle and  closed angle) are described on the appearance of the drainage  system within the eye, including as

## Function to extract text from PDF for respective headers

In [11]:
def extractTextPdf(pdf_path):
  
    document = pdf_path
    doc = fitz.open(document)
    #
    font_counts, styles = fonts(doc, granularity=False)
    #
    size_tag = font_tags(font_counts, styles)
    #
    elements = headers_para(doc, size_tag)
    print(elements)
    
    return elements


In [14]:
pdf_path = "/content/Published.SAFPJ.11.05.2017.pdf"
ele = extractTextPdf(pdf_path)

['<s8>See discussions, stats, and author profiles for this publication at:  https://www.researchgate.net/publication/316990409|', '', '<h2>The medical management of glaucoma|', '', '<s6>Article in South African Family Practice · May 2017|', '', '<s12>DOI: 10.4102/safp.v59i2.4669|', '', '<s10>CITATIONS|', '<s2>3|', '', '<s10>READS|', '<s2>11,056|', '', '<s6>2 authors:|', '<s6>Some of the authors of this publication are also working on these related projects:|', '<s6>Medical View project|', '<s6>Indigenous Knowledge Systems: Pharmacology and Toxicology View project|', '<s6>Andre Marais|', '<s6>University of Pretoria|', '', '<s8>20', '<s10>PUBLICATIONS', '<s8>34', '<s10>CITATIONS|', '<s10>SEE PROFILE|', '', '<s6>Elzbieta Osuch|', '<s6>Sefako Makgatho Health Sciences University|', '', '<s8>26', '<s10>PUBLICATIONS', '<s8>110', '<s10>CITATIONS|', '<s10>SEE PROFILE|', '', '<s6>All content following this page was uploaded by  Andre Marais  on 17 May 2017.|', '', '<s10>The user has requested en

In [ ]:
#Extract text for corresponding headers
    element_dict = {}
    for text in elements:
      #prev = text[0:start]
      if text.startswith("<h"): 
        start = text.index(">")
        header = text[start+1:].replace("|","")
    
      if text.startswith("<p") and header not in element_dict.keys():
        c = []
        start = text.index(">")
        c.append(text[start+1:].replace("|",""))
        element_dict[header] = c
      elif text.startswith("<p") and header in element_dict.keys():
        start = text.index(">")
        c.append(text[start+1:].replace("|",""))
        element_dict[header] =c
      else:
        continue
      
      for k,v in element_dict.items():
        p = ".".join([text.strip() for text in sent_tokenize(v[0])])
        rec = f"{k}:{p}\n"
        print(rec)
        with open('extracted_text.txt',"a+") as f:
          f.write(rec)




In [52]:
def header_text(edict):
  element_dict = {}
  for text in ele:
    #print(text)
    #prev = text[0:start]
    if text.startswith("<h"): 
      start = text.index(">")
      header = text[start+1:].replace("|","")
    
    if text.startswith("<p") and header not in element_dict.keys():
      c = []
      start = text.index(">")
      c.append(text[start+1:].replace("|",""))
      element_dict[header] = c
    elif text.startswith("<p") and header in element_dict.keys():
      start = text.index(">")
      c.append(text[start+1:].replace("|",""))
      element_dict[header] =c
    else:
      continue
  for k,v in element_dict.items():
        p = "".join(v)
        rec = f"{k}:{p}\n".strip()
        #print(rec)
        with open('Pdf_extracted_text.txt',"a+") as f:
          f.write(rec)
  return element_dict


In [53]:
element_dict = header_text(ele)

In [54]:
" ".join(element_dict['Conclusion '])

'General practitioners and family physicians play an important  role in the diagnosis and appropriate referral of patients with  suspected glaucoma. Fundoscopic examinations are quick and  easy to perform and are an opportunity that present at each  consultation. The early recognition of symptoms may improve  outcomes and delay progression. Taking a good medical history  and applying the principles of rational drug use may identify  potential pharmacological causes for glaucoma. Practitioners  prescribing any of the drugs known to cause acute angle glaucoma  should be vigilant in patients presenting with characteristic signs  or symptoms, and immediately refer them to an ophthalmologist.  Patients need to be counselled on the associated adverse effects  from glaucoma medication in order to improve adherence.      '

In [55]:
"".join(element_dict['Carbonic anhydrase inhibitors'])

'Acetazolamide, dorzolamide and brinzolamide are sulphonamide  derivatives  which  reduces  aqueous  humour  production  independent to its diuretic mode of action. Acetazolamide is  administered systemically in the form of oral tablets or intravenous  infusion, and additionally reduces the rate of aqueous inflow up to  50%. Its usefulness is limited to the management of acute closed  angle glaucoma due to unfavourable systemic side effects which  may include paresthesia, nausea, diarrhoea, loss of appetite and  dose related systemic acidosis. Topical application of dorzolamide and brinzolamide demonstrates an improvement in ocular blood  flow and long-term preservation of visual field. An average reduction  of  15–20% in baseline IOP can be expected. Side effects include  ocular irritation, dry eyes, blurred vision, eyelid inflammation and a  bitter taste.'

## Summarizing text based on the headers



In [18]:
sbio_tokenizer = AutoTokenizer.from_pretrained("t5-large")
sbio_model = AutoModelForSeq2SeqLM.from_pretrained("sumedh/biomedical_text_summarization")

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [30]:
sbio_tokenizer.model_max_length

512

In [ ]:
max_source_length = max([len(x) for x in inputs.input_ids])
print(f"Max source length: {max_source_length}")

In [23]:
for k,v in element_dict.items():
  sum = []
  #sumedh/biomedical_text_summarization
  tokenized_text = sbio_tokenizer.encode(v[0],return_tensors="pt",max_length=1024,truncation=True)
  summary = sbio_model.generate(tokenized_text,min_length=50,max_length=500)
  summary_text = sbio_tokenizer.decode(summary[0],skip_special_tokens=True)
  sum.append(".".join([text.strip().capitalize() for text in summary_text.split(".")]))


/usr/local/lib/python3.8/dist-packages/transformers/modeling_utils.py:764: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [26]:
sentences = []
for k,v in element_dict.items():
  sentences.append(v[0])

In [27]:
sentences

['Glaucoma is a mixed group of eye disorders sharing common  features which typically includes optic disc cupping and visual  field loss, thereby more accurately defined as an optic neuropathy.  Optic nerve damage results in progressive retinal ganglionic  cell loss and irreversible blindness, if not treated.',
 'Glaucoma is one of the leading causes of blindness worldwide. The main characteristic features include optic disc cupping, visual field  loss, and in most instances a raised intraocular pressure. Primary open angle glaucoma accounts for nearly two thirds of all cases, while  angle closure glaucoma contributes the majority of the remainder. Screening for glaucoma is not cost-effective, and diagnosis of the  primary open angle type is usually an incidental finding during ophthalmological examination. Various treatment options, including  medical and surgical interventions are accessible in the management of glaucoma. This article contains a brief description on the  pathophysiol

In [66]:
final_sum = []
" ".join(element_dict['Conclusion '])
for header,sentence in element_dict.items():
  sent = " ".join(element_dict[header])
  print(sent)
  #Sumedh Biomedical
  '''
  tokenized_text = sbio_tokenizer.encode(sent,return_tensors="pt")
  max_source_length = max([len(x) for x in tokenized_text])
  print(f"Max source length: {max_source_length}")
  malen = max_source_length + 50
  summary = sbio_model.generate(tokenized_text)
  summary_text = sbio_tokenizer.decode(summary[0],skip_special_tokens=True)
  '''
  # t5-small-finetuned-bioMedv2
  tokenized_text = tokenizer.encode(sent,return_tensors="pt",max_length=1024,truncation=True)
  max_source_length = max([len(x) for x in tokenized_text])
  print(f"Max source length: {max_source_length}")
  summary = model.generate(tokenized_text,min_length=30,max_length=800)
  summary_text = tokenizer.decode(summary[0],skip_special_tokens=True)
  summary_text.replace(":","")
  prompt = header+":"+summary_text
  final_sum.append(prompt)

Glaucoma is a mixed group of eye disorders sharing common  features which typically includes optic disc cupping and visual  field loss, thereby more accurately defined as an optic neuropathy.  Optic nerve damage results in progressive retinal ganglionic  cell loss and irreversible blindness, if not treated.  Traditionally,  glaucoma has been characterized by the existence of raised  intraocular pressure (IOP), however nerve damage may occur in the  presence of values within the statistically normal range between   8–21 mmHg. Glaucomatous disc and field changes in the absence  of elevated pressure constitutes normal or low tension glaucoma  (LTG).  Similarly, conditions exceeding the upper limit of 21 mmHg  may not be associated with signs of visual field loss or optic disc  damage and comprises ocular hypertension.   In addition to optic  neuropathy, the two main types of glaucoma (open angle and  closed angle) are described on the appearance of the drainage  system within the eye, inc

In [67]:
final_sum

['Introduction:Glaucoma (glaucoma) is a mixed group of eye disorders which share common features which typically includes optic disc cupping and visual field loss.',
 'Abstract:The pathophysiology of glaucoma is a key factor in blindness, and it is a common problem in the treatment of blindness.',
 'Corresponding author, email: dramarais@gmail.com / andre.marais@up.ac.za:Glaucoma is the most common cause of irreversible blindness in Africa (4.2%), and closed angle glaucoma is the highest in Africa (4.2%), while closed angle glaucoma is the highest in Asia (1.09%).',
 'Pathophysiology of glaucoma:Aqueous humour (a low protein containing fluid) is secreted by the ciliary epithelium providing structural support to the lens.',
 'Open angle glaucoma:The progression from glaucoma to chronic angle-closure Glaucoma (CACG) is asymptomatic until central vision is affected, resulting in progressive impairment of drainage.',
 'Closed angle glaucoma:Acute angle closure glaucoma (AACG) is the most c

In [68]:
with open("summary_t5_bioMedv2.txt","w+") as f:
  for item in final_sum:
    f.write(item)
    f.write("\n")

## Bert Summarizer

In [ ]:
from summarizer import Summarizer
model = Summarizer()

summary_length = int(len(sentence_tokens)*reduction_rate)
print(summary_length)
result = model(processed_text, num_sentences=summary_length, min_length=400)

print(''.join(result))

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


10
Compression and narrowing of  the canal reduces or obstructs the outflow of aqueous humour in  passing through the pupil, therefore accumulating in the anterior  chamber (Figure 1).Primary angle closure glaucoma (PACG) is  less common than POAG and accounts for approximately a third  of all cases, with a global incidence of 0.5%.Symptoms of closed  angle glaucoma depend on the degree of angle closure and in  some instances may resolve spontaneously; in which case it may  become chronic. Acute attacks are characterized  by intermittent symptoms such as severe eye pain, nausea and  vomiting, loss of vision, halos around lights, unilateral red tearing  eye, oval shaped non-reactive pupil, a cloudy cornea and the loss  of red reflex (Figure 3).In addition, it is estimated that prescribed, or over-the-counter  pharmacological agents are responsible for at least 30% of acute  angle closure glaucoma.


In [ ]:
sents = sent_tokenize(processed_text)
len(sents)

2

## Flan-t5-base Summarizer

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

In [ ]:
prompt_lenght = len(tokenizer(prompt.format(input=""))["input_ids"])
max_sample_length = tokenizer.model_max_length - prompt_lenght
print(f"Prompt length: {prompt_lenght}")
print(f"Max input length: {max_sample_length}")

Prompt length: 4140
Max input length: -3628


In [ ]:
tokenizer.model_max_length

512

In [ ]:
#generate prompt
prompt = f"Text Summarization : {processed_text}"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
max_source_length = max([len(x) for x in input_ids])
print(f"Max source length: {max_source_length}")

max_source_length = max([len(x) for x in input_ids])
max_source_length = min(max_source_length, max_sample_length)
print(f"Max source length: {max_source_length}")


Max source length: 4140
Max source length: -3628


In [ ]:
outputs = model.generate(input_ids,max_length=400)
#
sum = tokenizer.decode(outputs[0],skip_special_tokens=True)
sum

'Understand the pathophysiology of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand

## Automatic Tet summarizer

In [ ]:
! pip install sumy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 KB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 95.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21714 sha256=d3a9c43f5eda5390a23529a7abd18e611a26dcbbde22bbfd16991f5e4a9aca42
  Stored in directory: /root/.cache/pip/wheels/5f/0d/0c/2062d8c1758b4b1a2e42b4a63e6660d9ec2ba9463cfee9eeab
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681845 sha256=210ee5f29fb1217d411531d726d400bde8779b71d6f9eff90da4c2e444dd737b
  Stored in directory: /root/.cache/pip/wheels/e2/aa/0f/c224e473b464387170b83ca7c66947b4a7e33e8d903a679748
Successfully built

In [ ]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer as Summarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
methods = ["LSA", "luhn", "edmundson", "text-rank", "lex-rank", "random", "reduction", "kl-sum"]

supported_languages = ["english", "french", "arabic", "chinese", "czech", "german", "italian", "hebrew", 
                        "japanese", "portuguese", "slovak", "spanish", "ukrainian", "greek"]

In [ ]:
parser = PlaintextParser.from_string(processed_text, Tokenizer("english"))

In [ ]:


LANGUAGE = "english"
SENTENCES_COUNT = 105
stemmer = Stemmer(LANGUAGE)

summarizer = Summarizer(stemmer)
summarizer.stop_words = get_stop_words(LANGUAGE)

sum = []
for sentence in summarizer(parser.document, SENTENCES_COUNT):
        print(sentence)
       

Glaucoma is a mixed group of eye disorders sharing common  features which typically includes optic disc cupping and visual  field loss, thereby more accurately defined as an optic neuropathy.Optic nerve damage results in progressive retinal ganglionic  cell loss and irreversible blindness, if not treated.Traditionally,  glaucoma has been characterized by the existence of raised  intraocular pressure (IOP), however nerve damage may occur in the  presence of values within the statistically normal range between   8–21 mmHg.Glaucomatous disc and field changes in the absence  of elevated pressure constitutes normal or low tension glaucoma  (LTG).Similarly, conditions exceeding the upper limit of 21 mmHg  may not be associated with signs of visual field loss or optic disc  damage and comprises ocular hypertension.In addition to optic  neuropathy, the two main types of glaucoma (open angle and  closed angle) are described on the appearance of the drainage  system within the eye, including ass

In [ ]:
processed_text

'Glaucoma is a mixed group of eye disorders sharing common  features which typically includes optic disc cupping and visual  field loss, thereby more accurately defined as an optic neuropathy.Optic nerve damage results in progressive retinal ganglionic  cell loss and irreversible blindness, if not treated.Traditionally,  glaucoma has been characterized by the existence of raised  intraocular pressure (IOP), however nerve damage may occur in the  presence of values within the statistically normal range between   8–21 mmHg.Glaucomatous disc and field changes in the absence  of elevated pressure constitutes normal or low tension glaucoma  (LTG).Similarly, conditions exceeding the upper limit of 21 mmHg  may not be associated with signs of visual field loss or optic disc  damage and comprises ocular hypertension.In addition to optic  neuropathy, the two main types of glaucoma (open angle and  closed angle) are described on the appearance of the drainage  system within the eye, including as

In [ ]:
sum.strip()

'Glaucoma is a mixed group of eye disorders sharing common  features which typically includes optic disc cupping and visual  field loss, thereby more accurately defined as an optic neuropathy.Optic nerve damage results in progressive retinal ganglionic  cell loss and irreversible blindness, if not treated.Traditionally,  glaucoma has been characterized by the existence of raised  intraocular pressure (IOP), however nerve damage may occur in the  presence of values within the statistically normal range between   8–21 mmHg.Glaucomatous disc and field changes in the absence  of elevated pressure constitutes normal or low tension glaucoma  (LTG).Similarly, conditions exceeding the upper limit of 21 mmHg  may not be associated with signs of visual field loss or optic disc  damage and comprises ocular hypertension.In addition to optic  neuropathy, the two main types of glaucoma (open angle and  closed angle) are described on the appearance of the drainage  system within the eye, including as

## Summart Text

In [ ]:
sum = """ Glaucoma is a mixed group of eye disorders sharing common  features which typically includes optic disc cupping and visual  field loss, thereby more accurately defined as an optic neuropathy.Optic nerve damage results in progressive retinal ganglionic  cell loss and irreversible blindness, if not treated.Traditionally,  glaucoma has been characterized by the existence of raised  intraocular pressure (IOP), however nerve damage may occur in the  presence of values within the statistically normal range between   8–21 mmHg.Glaucomatous disc and field changes in the absence  of elevated pressure constitutes normal or low tension glaucoma  (LTG).Similarly, conditions exceeding the upper limit of 21 mmHg  may not be associated with signs of visual field loss or optic disc  damage and comprises ocular hypertension.In addition to optic  neuropathy, the two main types of glaucoma (open angle and  closed angle) are described on the appearance of the drainage  system within the eye, including associated increase in the IOP  (Figure 1).Glaucoma is one of the leading causes of blindness worldwide.The main characteristic features include optic disc cupping, visual field  loss, and in most instances a raised intraocular pressure.Primary open angle glaucoma accounts for nearly two thirds of all cases, while  angle closure glaucoma contributes the majority of the remainder.Screening for glaucoma is not cost-effective, and diagnosis of the  primary open angle type is usually an incidental finding during ophthalmological examination.Various treatment options, including  medical and surgical interventions are accessible in the management of glaucoma.This article contains a brief description on the  pathophysiology and focuses on the pharmacological options available in the treatment of glaucoma.Keywords:  Glaucoma, prostaglandin analogs, carbonic anhydrase inhibitors, beta antagonists, alpha 2 adrenergic agonists, cholinergic  agonists Both types can be further divided into primary versus secondary,  and congenital versus acquired.Glaucoma has a global prevalence  of 3.54% in those aged between 40 and 80 years, and is the  second leading cause of irreversible blindness, only superseded  by cataracts.Open angle glaucoma is the highest in Africa (4.2%),  while closed angle glaucoma is more prevalent in Asia (1.09%).Aqueous humour (a transparent, low protein containing fluid) is  continuously secreted by the ciliary epithelium providing structural  support to the lens.This watery fluid is present in both the anterior  and posterior chambers of the eye and drains through a sieve-like  trabecular meshwork located in the angle between the cornea  and the iris.Final drainage out of the eye occurs via the canal of  Schlemm  into the episcleral vein or intrascleral venous plexus  terminating in the ocular veins.In open angle glaucoma, the trabecular meshwork undergoes  morphological changes over time, resulting in progressive  impairment of drainage although it remains anatomically open. (Figure 1) Open angle glaucoma most often affects both eyes,  but asymmetric involvement is not uncommon.Primary open  angle glaucoma (POAG) is the most common type and accounts  for 74% of all glaucoma cases.It is estimated that 57.5 million  people (3.05%) worldwide suffer from POAG.Prevalence among  Black populations is the highest, with 2.1% at age 40 and 12.2% at  age 80 being reported.The incidence in Asian populations (0.8%  at age 40 and 4.3% at age 80), and Caucasians (0.4% at 40 years  and 5.3% at 80 years) suggests an ethno-genetic disease pattern.Although the exact cause of POAG remains unknown, more than  fifty risk factors and secondary causes have been identified.Some of these include diabetes mellitus, migraine headache,  cardiovascular disease, asthma or COPD (inhaled anticholinergics  and glucocorticosteroids), myopia, advanced age and a family  history of first-degree relatives with the same condition.Disease  progression leads to the formation of scar tissue between the iris  and trabecular meshwork and will ultimately result in closing off  the anterior chamber.The consequent rise in IOP is transmitted to  the optic disc and is eventually responsible for atrophy and nerve  fiber damage.Owing to the slow progression of the disease, POAG  remains asymptomatic until central vision is affected.It does not  present with headache, eye pain or loss of visual acuity.Diagnosis  is usually incidental during routine funduscopic or optometry  examinations.A cup : disc ratio of 0.6 (glaucomatous cupping), and  central corneal thickness in the presence of risk factors is suggestive  of glaucoma, and requires ophthalmological referral (Figure 2).The progression from POAG to Chronic Angle-Closure Glaucoma  (CACG), requires lifelong treatment.Official general population screening policies do not exist and are  not cost-effective.Measuring IOP has a sensitivity of only 47.1%,  implying that nearly half of patients with POAG will have IOP below  22 mmHg.However, patients with risk factors are encouraged to  undergo annual ophthalmological examinations and additional  IOP measurements.With closed angle glaucoma, a primary hypermetropic anatomical  defect (such as weakened ciliary muscles, naturally small eyes,  narrow angle or thick lens) in the absence of an identifiable  secondary cause, results in the pupil compressing the drainage  canal between the iris and cornea.Compression and narrowing of  the canal reduces or obstructs the outflow of aqueous humour in  passing through the pupil, therefore accumulating in the anterior  chamber (Figure 1).Primary angle closure glaucoma (PACG) is  less common than POAG and accounts for approximately a third  of all cases, with a global incidence of 0.5%.Symptoms of closed  angle glaucoma depend on the degree of angle closure and in  some instances may resolve spontaneously; in which case it may  become chronic.In established disease, vision loss is severe and  permanent.Secondary causes of angle-closure glaucoma include  masses or hemorrhage in the posterior segment of the eyeball or  the presence of a fibrovascular membrane (neovascular glaucoma)  over the anterior chamber angle.Risk factors for angle closure  glaucoma include older age (> 60 years), female gender, Asian  ethnicity, hyperopia and a positive family history.Attacks occur  more frequently at night when diminished light cause mydriasis and  narrowing of the angle.Unexpected closure (acute angle closure  glaucoma, AACG) of the anterior chamber angle due to pupillary  block may result in a sudden rise in IOP.It constitutes a medical  emergency, since permanent blindness and irreversible damage  to the optic nerve may occur.Acute attacks are characterized  by intermittent symptoms such as severe eye pain, nausea and  vomiting, loss of vision, halos around lights, unilateral red tearing  eye, oval shaped non-reactive pupil, a cloudy cornea and the loss  of red reflex (Figure 3).In addition, it is estimated that prescribed, or over-the-counter  pharmacological agents are responsible for at least 30% of acute  angle closure glaucoma.These include antihistamines, cholinergic-  and anticholinergic agents, adrenergic agents, antidepressants,  sulphonamides, anticoagulants and glucocorticosteroids.Table 1  below summarises the mechanism responsible for causing AACG.The goal in treating POAG is to establish and maintain the intra- ocular pressure at a range where visual field loss will have the  least negative impact on the patient’s perceived visual disability.“Various studies have shown that visual field loss in glaucoma has  been  associated with a significant decrease in a person’s vision- related quality of life.This is evident in activities requiring optimal  vision, such as dark adaptation, avoiding unintentional bumping  into objects and outdoor mobility.Determining the target  intraocular pressure needs to be individualised for each patient,  since the measured pressure at diagnosis could be within the normal  range of 8 to 21 mmHg in 30–50% of patients.Aiming for pressures  within these ranges will contribute towards optic nerve damage  and further progression.It is generally accepted that patients with  early disease benefit from a reduction of at least 20% in the baseline  value.A reduction of 30% is recommended in individuals with  more advanced features of POAG.Target IOP should continuously  be re-calculated during monitoring and follow-up assessment  of visual fields and disc and nerve fiber examinations.Reducing  the IOP, however, does not guarantee that progression will not  transpire.The choice of therapy depends on the relative risks and  benefits of available modalities, which include pharmacological  treatment, laser therapy or incisional surgery.Factors such as cost,  dosing, patient’s age, the presence of comorbidities (asthma or  COPD), pregnancy, adherence to treatment and the degree of optic  disc damage should be considered in choosing an appropriate  treatment regimen.In treating AACG, the primary goal is to make the correct diagnosis to  prevent permanent loss of vision.This is achieved by rapid lowering  of the IOP and reversal of the angle closure.Empiric treatment with  topical and systemic aqueous humour suppressants (β blockers  and acetazolamide) reduces the IOP, followed by pilocarpine to  reverse the pupil block.In the absence of available trials comparing  medical options for treatment of AACG, the initial management  usually consists of administering one drop of 0.5% timolol,   1% apraclonidine and 2% pilocarpine one minute apart.This is  followed by intravenous acetazolamide or mannitol.Eye pressure  should be checked every hour until specialist ophthalmological  care can be provided.Pharmacological management is the most common initial  intervention in lowering the IOP, although laser trabeculoplasty  and surgery are frequently used to slow the disease progression.Surgical treatment has no advantage over pharmacological  management in visual field protection, however surgical  interventions attain target IOP in 46% of cases compared to 35% in  those being treated medically.Several agents are available and the primary choice of drug depends  on the degree of pressure lowering which needs to be achieved.It  is recommended to initiate treatment on one type of medication.If an inadequate response in lowering the IOP is observed, the  dose may be adjusted, or a second agent may be added.If the  first agent fails to demonstrate any measureable clinical response,  it should be removed from the regimen and be replaced with an  alternative drug.Pharmacological agents used to lower the IOP  include prostaglandin analogs, carbonic anhydrase inhibitors,  α  adrenergic agonists, cholinergic agonists, β blockers and  hyperosmotic agents.Adequate treatment of glaucoma requires  a high level of adherence.It is estimated that 45% of patients will  default on chronic treatment, if they are not appropriately followed  up and counselled regarding dosage instructions and possible side  effects.Table 2 below summarises the currently available drugs in  South Africa used to lower intraocular pressure.Latanoprost, travoprost and bimatoprost are prostaglandin   F agonists, which increases the uveoscleral outflow of aqueous  humour.It is considered the most potent group of topical ocular  hypotensive agents currently available and may lower baseline IOP  by 25–35%.There are few associated systemic side effects and  mostly relates to headaches.In addition, latanoprost may cause  joint/muscle pain and flu-like symptoms.Local adverse effects  include blurred vision, conjunctival hyperaemia, eye discomfort,  permanent iridial pigmentation, macular edema and thickening of  the eyelashes.Acetazolamide, dorzolamide and brinzolamide are sulphonamide  derivatives  which  reduces  aqueous  humour  production  independent to its diuretic mode of action.Acetazolamide is  administered systemically in the form of oral tablets or intravenous  infusion, and additionally reduces the rate of aqueous inflow up to  50%.Its usefulness is limited to the management of acute closed  angle glaucoma due to unfavourable systemic side effects which  may include paresthesia, nausea, diarrhoea, loss of appetite and  dose related systemic acidosis.Topical application of dorzolamide  and brinzolamide demonstrates an improvement in ocular blood  flow and long-term preservation of visual field.An average reduction  of  15–20% in baseline IOP can be expected.Side effects include  ocular irritation, dry eyes, blurred vision, eyelid inflammation and a  bitter taste.Brimonidine is a selective α2 receptor agonist.It initially reduces  aqueous humour production followed by a subsequent increase  in the uveoscleral outflow, therefore proving to be useful in  preventing a rise in IOP prior to anterior segment laser surgery.Brimonidine reduces baseline IOP with approximately 20%.Apraclonidine reduces the formation of aqueous humour but has  no effect on the facilitation of outflow.It has affinity for both α1  and α2 adreno-receptors and are used to prevent postsurgical  elevation of IOP after anterior segment laser therapy.Apraclonidine  reduces baseline IOP by 30% and is additionally used as adjuvant  therapy in resistant cases of glaucoma.Systemic side effects of the  alpha agonists are related to the central nervous system (headache,  fatigue, insomnia, depression) and respiratory system.Caution  should be taken in patients with cerebral or coronary insufficiency,  postural hypotension and renal or hepatic failure.Hypersensitivity  reactions, ocular irritation, eyelid edema, foreign body sensation  and dry eyes are common with topical use.Pilocarpine is the only available cholinergic agent in the  management of acute closed-angle glaucoma registered in South  Africa.Administration results in the increase in aqueous humour  outflow by contraction of the ciliary muscle and constriction of the  pupil.Poorly tolerated side effects, such as ciliary muscle spasm,  myopia and decreased vision limits its usefulness.Although prostaglandin receptor agonists are the most potent  agents in reducing the IOP, beta blockers are regarded as the first  choice in treating open angle glaucoma.Individual agents differ in  their ability to lower IOP, with betaxolol achieving a 15% decrease,  compared to 20–25% with timolol and levobunolol respectively.Betaxolol is, however, more cardioselective (β1) and therefore has  fewer pulmonary side effects compared to timolol and levobunolol  (which are non-selective antagonists on β1 and β2 receptors).The  mechanism involves a reduction of aqueous humour production  and does not cause miosis or accommodation disturbances  compared to cholinergic agents.The most common adverse effects  are ocular irritation and dry eyes.All the β blockers (including the  cardioselective agents) are contraindicated in patients with asthma,  COPD and bradycardia, unless alternative treatment is unavailable.Several combination agents are available in the treatment of  glaucoma.Most of these combinations include timolol, since  β blockers are still considered first line therapy in open-angle  glaucoma and ocular hypertension.Fixed-dose combination  therapy is more efficacious than its individual components, but still  less efficacious than its respective unfixed combinations.It provides  for simplified treatment regimes and enhances compliance.Fixed  dose combinations containing prostaglandins have a lower risk  in causing hyperemia.Combination of timolol with dorzolamide  is superior in visual field preservation compared to timolol and  brinzolamide.Combination preparations with brimonidine  have less visual disturbances, taste alterations and ocular allergic  reactions compared to the single ingredient product.With the recent attention cannabis has been receiving in the  management of pain and other cancer related conditions, it  is important to remember the favourable effect of its active  ingredient, Δ -tetrahydrocannabinol (THC), in reducing ocular  blood flow.Studies from the 1970s have reported a 30% reduction  in the IOP by inhalation or intravenous administration of THC.The  use is, however, limited owing to the cardiovascular (increased heart  rate and decreased blood pressure) and neurological (tolerance  and addiction) side effects.Topical application does not produce  a significant clinical effect.The use of synthetic cannabinoids may  provide a basis for potential future treatment strategies.General practitioners and family physicians play an important  role in the diagnosis and appropriate referral of patients with  suspected glaucoma.Fundoscopic examinations are quick and  easy to perform and are an opportunity that present at each  consultation.The early recognition of symptoms may improve  outcomes and delay progression.Taking a good medical history  and applying the principles of rational drug use may identify  potential pharmacological causes for glaucoma.Practitioners  prescribing any of the drugs known to cause acute angle glaucoma  should be vigilant in patients presenting with characteristic signs  or symptoms, and immediately refer them to an ophthalmologist.Patients need to be counselled on the associated adverse effects  from glaucoma medication in order to improve adherence.
"""

In [ ]:
def summarize(method, language, sentence_count, input_type, input_):
  if method== 'LSA':
    from sumy.summarizers.lsa import LsaSummarizer as Summarizer
  if method=='text-rank':
    from sumy.summarizers.text_rank import TextRankSummarizer as Summarizer
  if method=='lex-rank':
    from sumy.summarizers.lex_rank import LexRankSummarizer as Summarizer
  if method=='edmundson':
    from sumy.summarizers.edmundson import EdmundsonSummarizer as Summarizer
  if method=='luhn':
    from sumy.summarizers.luhn import LuhnSummarizer as Summarizer
  if method=='kl-sum':
    from sumy.summarizers.kl import KLSummarizer as Summarizer
  if method=='random':
    from sumy.summarizers.random import RandomSummarizer as Summarizer
  if method=='reduction':
    from sumy.summarizers.reduction import ReductionSummarizer as Summarizer

  if input_type=="URL":
    parser = HtmlParser.from_url(input_, Tokenizer(language))
  if input_type=="text":
    parser = PlaintextParser.from_string(input_, Tokenizer(language))
    
  stemmer = Stemmer(language)
  summarizer = Summarizer(stemmer)
  stop_words = get_stop_words(language)

  if method=='edmundson':
    summarizer.null_words = stop_words
    summarizer.bonus_words = parser.significant_words
    summarizer.stigma_words = parser.stigma_words
  else:
    summarizer.stop_words = stop_words

  summary_sentences = summarizer(parser.document, sentence_count)
  summary = ' '.join([str(sentence) for sentence in summary_sentences])
  
  return summary

In [ ]:
input_type ="text"
language = "english"
sentence_count = 5
input_ = processed_text
for method in methods:
  result = summarize(method, language, sentence_count, input_type, input_)
  print(f"{method} : {result}")
  print("\n")

LSA : Glaucoma is a mixed group of eye disorders sharing common  features which typically includes optic disc cupping and visual  field loss, thereby more accurately defined as an optic neuropathy.Optic nerve damage results in progressive retinal ganglionic  cell loss and irreversible blindness, if not treated.Traditionally,  glaucoma has been characterized by the existence of raised  intraocular pressure (IOP), however nerve damage may occur in the  presence of values within the statistically normal range between   8–21 mmHg.Glaucomatous disc and field changes in the absence  of elevated pressure constitutes normal or low tension glaucoma  (LTG).Similarly, conditions exceeding the upper limit of 21 mmHg  may not be associated with signs of visual field loss or optic disc  damage and comprises ocular hypertension.In addition to optic  neuropathy, the two main types of glaucoma (open angle and  closed angle) are described on the appearance of the drainage  system within the eye, includi

In [5]:
input_ = """Glaucoma is a mixed group of eye disorders sharing common  features which typically includes optic disc cupping and visual  field loss, thereby more accurately defined as an optic neuropathy.Optic nerve damage results in progressive retinal ganglionic  cell loss and irreversible blindness, if not treated.Traditionally,  glaucoma has been characterized by the existence of raised  intraocular pressure (IOP), however nerve damage may occur in the  presence of values within the statistically normal range between   8–21 mmHg"""

In [ ]:
summarize('reduction', language, sentence_count, input_type, input_)

'Glaucoma is a mixed group of eye disorders sharing common  features which typically includes optic disc cupping and visual  field loss, thereby more accurately defined as an optic neuropathy.Optic nerve damage results in progressive retinal ganglionic  cell loss and irreversible blindness, if not treated.Traditionally,  glaucoma has been characterized by the existence of raised  intraocular pressure (IOP), however nerve damage may occur in the  presence of values within the statistically normal range between   8–21 mmHg'

In [ ]:
processed_text

'Glaucoma is a mixed group of eye disorders sharing common  features which typically includes optic disc cupping and visual  field loss, thereby more accurately defined as an optic neuropathy.Optic nerve damage results in progressive retinal ganglionic  cell loss and irreversible blindness, if not treated.Traditionally,  glaucoma has been characterized by the existence of raised  intraocular pressure (IOP), however nerve damage may occur in the  presence of values within the statistically normal range between   8–21 mmHg.Glaucomatous disc and field changes in the absence  of elevated pressure constitutes normal or low tension glaucoma  (LTG).Similarly, conditions exceeding the upper limit of 21 mmHg  may not be associated with signs of visual field loss or optic disc  damage and comprises ocular hypertension.In addition to optic  neuropathy, the two main types of glaucoma (open angle and  closed angle) are described on the appearance of the drainage  system within the eye, including as

## Bio Medical Text Summarization using huggingface model card : **sumedh/biomedical_text_summarization**

In [132]:
sbio_tokenizer = AutoTokenizer.from_pretrained("t5-large")
sbio_model = AutoModelForSeq2SeqLM.from_pretrained("sumedh/biomedical_text_summarization")

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [6]:
tokenized_text = tokenizer.encode(input_,return_tensors="pt",max_length=1024,truncation=True)
summary = model.generate(tokenized_text,min_length=30,max_length=500)
summary_text = tokenizer.decode(summary[0],skip_special_tokens=True)
".".join([text.strip().capitalize() for text in summary_text.split(".")])

/usr/local/lib/python3.8/dist-packages/transformers/modeling_utils.py:764: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


'Glaucoma is a mixed group of eye disorders sharing common features which typically includes optic disc cupping and visual field loss, thereby more accurately defined as an optic neuropathy.The aim of this study was to evaluate the effect of intraocular pressure on nerve damage in patients with retinal ganglionic cell loss and irreversible blindness.We performed a comparative analysis of peripheral blood mononuclear cells from two groups of eyes that were treated with corticosteroids for three consecutive days using a combination of electrophoresis'

**Save the summary generated into a text into a text file**

In [7]:
with open('sumedh_biomedical.txt',"a+") as f:
  f.write(".".join([text.strip().capitalize() for text in summary_text.split(".")]))

## Model Card : PeterBanning71/t5-small-finetuned-bioMedv1

In [8]:
tokenizer = AutoTokenizer.from_pretrained("PeterBanning71/t5-small-finetuned-bioMedv1")
model = AutoModelForSeq2SeqLM.from_pretrained("PeterBanning71/t5-small-finetuned-bioMedv1")

In [18]:
prompt = "Summarization:"+input_

In [22]:
tokenized_text = tokenizer.encode(prompt,return_tensors="pt",max_length=1024,truncation=True)
summary = model.generate(tokenized_text,min_length=30,max_length=500)
summary_text = tokenizer.decode(summary[0],skip_special_tokens=True)
summary_text.replace(":","")

'Glaucoma is a mixed group of eye disorders sharing common features which typically includes optic disc cupping and visual field loss, thereby more accurately defined as an optic neuropathy.Optic nerve damage results in progressive retinal ganglionic cell loss and irreversible blindness, if not treated.'

In [23]:
with open('PeterBaning_biomedical.txt',"a+") as f:
  f.write(summary_text.replace(":",""))

## Model Card : PeterBanning71/**t5-small-finetuned-bioMedv2**

In [131]:
bioMedv2_tokenizer = AutoTokenizer.from_pretrained("PeterBanning71/t5-small-finetuned-xsum-finetuned-bioMedv2")
bioMedv2_model = AutoModelForSeq2SeqLM.from_pretrained("PeterBanning71/t5-small-finetuned-xsum-finetuned-bioMedv2")

In [28]:
tokenized_text = tokenizer.encode(input_,return_tensors="pt",max_length=1024,truncation=True)
summary = model.generate(tokenized_text,min_length=30,max_length=500)
summary_text = tokenizer.decode(summary[0],skip_special_tokens=True)
summary_text.replace(":","")

'Glaucoma is a mixed group of eye disorders which share common features which typically includes optic disc cupping and visual field loss, thereby more accurately defined as an optic neuropathy.'

In [29]:
with open('PeterBaning_biomedical_v2.txt',"a+") as f:
  f.write(summary_text.replace(":",""))

## Model Card : PeterBanning71/**t5-small-finetuned-bioMedv3**

In [65]:
tokenizer = AutoTokenizer.from_pretrained("PeterBanning71/t5-small-finetuned-xsum-finetuned-bioMedv3")
model = AutoModelForSeq2SeqLM.from_pretrained("PeterBanning71/t5-small-finetuned-xsum-finetuned-bioMedv3")

In [25]:
tokenized_text = tokenizer.encode(input_,return_tensors="pt",max_length=1024,truncation=True)
summary = model.generate(tokenized_text,min_length=30,max_length=500)
summary_text = tokenizer.decode(summary[0],skip_special_tokens=True)
summary_text.replace(":","")

'Glaucoma is a mixed group of eye disorders which share common features which typically includes optic disc cupping and visual field loss, thereby more accurately defined as an optic neuropathy.'

In [26]:
with open('PeterBaning_biomedical_v3.txt',"a+") as f:
  f.write(summary_text.replace(":",""))

## Auto train Biomedical Model

In [30]:
tokenizer = AutoTokenizer.from_pretrained("L-macc/autotrain-Biomedical_sc_summ-1217846142")
model = AutoModelForSeq2SeqLM.from_pretrained("L-macc/autotrain-Biomedical_sc_summ-1217846142")

In [34]:
tokenized_text = tokenizer.encode(input_,return_tensors="pt",max_length=1024,truncation=True)
summary = model.generate(tokenized_text,min_length=30,max_length=600)
summary_text = tokenizer.decode(summary[0],skip_special_tokens=True)
summary_text.replace(":","")

'glaucoma is a mixed group of eye disorders sharing common features which typically includes optic disc cupping and visual field loss, thereby more accurately defined as an optic neuropathy. optical nerve damage results in progressive retinal ganglionic cell loss and irreversible blindness, if not treated. we report here on the case of a patient with glaococcus who presented with increased intraocular pressure due to elevated intra'

In [32]:
summary_text

'glaucoma is a mixed group of eye disorders sharing common features which typically includes optic disc cupping and visual field loss, thereby more accurately defined as an optic neuropathy. optical nerve damage results in progressive retinal ganglionic cell loss and irreversible blindness, if not treated. we report here on the case of a patient with glaococcus who presented with increased intraocular pressure due to elevated intra'

In [33]:
".".join([text.strip().capitalize() for text in summary_text.split(".")])

'Glaucoma is a mixed group of eye disorders sharing common features which typically includes optic disc cupping and visual field loss, thereby more accurately defined as an optic neuropathy.Optical nerve damage results in progressive retinal ganglionic cell loss and irreversible blindness, if not treated.We report here on the case of a patient with glaococcus who presented with increased intraocular pressure due to elevated intra'

## google/bigbird-pegasus-large-arxiv

In [37]:
from transformers import BigBirdPegasusForConditionalGeneration, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-arxiv")

# by default encoder-attention is `block_sparse` with num_random_blocks=3, block_size=64
model = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-arxiv")

# decoder attention type can't be changed & will be "original_full"
# you can change `attention_type` (encoder only) to full attention like this:
#model = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-arxiv", attention_type="original_full")

# you can change `block_size` & `num_random_blocks` like this:
model = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-arxiv", block_size=16, num_random_blocks=2)

inputs = tokenizer(input_, return_tensors='pt',max_length=500)
prediction = model.generate(**inputs)
prediction = tokenizer.batch_decode(prediction,skip_special_tokens=True)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 256 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [41]:
# you can change `block_size` & `num_random_blocks` like this:
model = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-arxiv")
prompt = f"Text Summarization : {input_}"
inputs = tokenizer(input_, return_tensors='pt',max_length=500,truncation=True)
prediction = model.generate(**inputs)
prediction = tokenizer.batch_decode(prediction,skip_special_tokens=True)

Attention type 'block_sparse' is not possible if sequence_length: 94 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


In [49]:
tokenizer.model_max_length

4096

In [46]:
max_source_length = max([len(x) for x in inputs.input_ids])
print(f"Max source length: {max_source_length}")

Max source length: 100


In [48]:
prediction

['in the case of blindness in a mixed group of optic sharing features, the presence of pressure in the visual field loss has been characterized. in the case of blindness in a mixed group of optic sharing features, the presence of pressure in the visual field loss has been characterized. in this paper, we have characterized the presence of pressure in the visual field loss in a mixed group of optic sharing features. <n> we have characterized the presence of pressure in the visual field loss in a mixed group of optic sharing features. <n> we have characterized the presence of pressure in the visual field loss in a mixed group of optic sharing features. <n> we have characterized the presence of pressure in the visual field loss in a mixed group of optic sharing features. <n> we have characterized the presence of pressure in the visual field loss in a mixed group of optic sharing features. <n> we have characterized the presence of pressure in the visual field loss in a mixed group of optic

## Long text summarization

## Haystack

In [ ]:
%%bash
pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,ocr]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-llezqpsl/farm-haystack_b44fd738586d41acbc2bf0733b356e07
  Resolved https://github.com/deepset-ai/haystack.git to commit 44509cd6a1db8deefa9dfb9dfbeb6a058c03cc7c
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.4/228.4 KB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 11.1 MB/s eta 0

  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-llezqpsl/farm-haystack_b44fd738586d41acbc2bf0733b356e07


In [ ]:
# For Colab/linux based machines:
!wget https://dl.xpdfreader.com/xpdf-tools-linux-4.04.tar.gz
!tar -xvf xpdf-tools-linux-4.04.tar.gz && sudo cp xpdf-tools-linux-4.04/bin64/pdftotext /usr/local/bin

--2023-02-17 16:51:07--  https://dl.xpdfreader.com/xpdf-tools-linux-4.04.tar.gz
Resolving dl.xpdfreader.com (dl.xpdfreader.com)... 45.79.72.155
Connecting to dl.xpdfreader.com (dl.xpdfreader.com)|45.79.72.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23687259 (23M) [application/x-gzip]
Saving to: ‘xpdf-tools-linux-4.04.tar.gz’

xpdf-tools-linux-4. 100%[===================>]  22.59M  8.88MB/s    in 2.5s    

2023-02-17 16:51:10 (8.88 MB/s) - ‘xpdf-tools-linux-4.04.tar.gz’ saved [23687259/23687259]

xpdf-tools-linux-4.04/
xpdf-tools-linux-4.04/CHANGES
xpdf-tools-linux-4.04/COPYING3
xpdf-tools-linux-4.04/INSTALL
xpdf-tools-linux-4.04/ANNOUNCE
xpdf-tools-linux-4.04/bin64/
xpdf-tools-linux-4.04/bin64/pdftopng
xpdf-tools-linux-4.04/bin64/pdftohtml
xpdf-tools-linux-4.04/bin64/pdfinfo
xpdf-tools-linux-4.04/bin64/pdffonts
xpdf-tools-linux-4.04/bin64/pdfimages
xpdf-tools-linux-4.04/bin64/pdftotext
xpdf-tools-linux-4.04/bin64/pdftoppm
xpdf-tools-linux-4.04/bin64/pd

In [ ]:
! pip install --ignore-installed Pillow==9.0.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached Pillow-9.0.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.3 MB)


In [ ]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [ ]:
! mkdir doc_dir


In [ ]:
import shutil
shutil.copy("/content/input1.pdf","/content/doc_dir")

'/content/doc_dir/input1.pdf'

In [ ]:
from haystack.nodes import TextConverter, PDFToTextConverter, DocxToTextConverter, PreProcessor

converter = PDFToTextConverter(remove_numeric_tables=True, valid_languages=["en"])
doc_pdf = converter.convert(file_path="/content/doc_dir/input1.pdf", meta=None)[0]


In [ ]:
from haystack.utils import convert_files_to_docs
from haystack.nodes import PreProcessor

ModuleNotFoundError: ignored

In [ ]:
doc_dir = "/content/doc_dir"
# convert_files_to_docs automatically applies the right converter to each file in a directory
all_docs = convert_files_to_docs(dir_path=doc_dir)
#  PreProcessor class is designed to help clean text and split text 
preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by="word",
    split_length=100,
    split_respect_sentence_boundary=True,
)
docs = preprocessor.process(all_docs)

print(f"n_files_input: {len(all_docs)}\nn_docs_output: {len(docs)}")

INFO:haystack.utils.preprocessing:Converting /content/doc_dir/input1.pdf


Preprocessing:   0%|          | 0/1 [00:00<?, ?docs/s]

n_files_input: 1
n_docs_output: 51


In [ ]:
docs[0].content

'See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/316990409\nThe medical management of glaucoma\nArticle in South African Family Practice · May 2017\nDOI: 10.4102/safp.v59i2.4669\nCITATIONS\n3\nREADS\n11,056\n2 authors:\nSome of the authors of this publication are also working on these related projects:\nMedical View project\nIndigenous Knowledge Systems: Pharmacology and Toxicology View project\nAndre Marais\nUniversity of Pretoria\n20 PUBLICATIONS 34 CITATIONS\nSEE PROFILE\nElzbieta Osuch\nSefako Makgatho Health Sciences University\n26 PUBLICATIONS 110 CITATIONS\nSEE PROFILE\nAll content following this page was uploaded by Andre Marais on 17 May 2017.\n'

In [ ]:
docs[3].content

'Glaucomatous disc and field changes in the absence\nof elevated pressure constitutes normal or low tension glaucoma\n(LTG).2 Similarly, conditions exceeding the upper limit of 21 mmHg\nmay not be associated with signs of visual field loss or optic disc\ndamage and comprises ocular hypertension.3 In addition to optic\nneuropathy, the two main types of glaucoma (open angle and\nclosed angle) are described on the appearance of the drainage\nsystem within the eye, including associated increase in the IOP\n(Figure 1).\nAbstract\nGlaucoma is one of the leading causes of blindness worldwide. '